
# Analisi streaming HLL vs HLL++ (paper-oriented)

Questo notebook usa **solo risultati streaming** e segue un'impostazione coerente con i paper:
- confronto tra errore relativo di HLL e HLL++;
- individuazione delle regioni in cui HLL++ migliora sensibilmente;
- annotazione di soglie teoriche (`2.5*m`) e soglie pratiche HLL++ (`Threshold(p)` dall'appendice).

I grafici sono interattivi (zoom/pan, anche sull'asse Y) e con legenda ad alta leggibilità.


In [1]:

from pathlib import Path
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

PLOT_CONFIG = {
    'scrollZoom': True,
    'displaylogo': False,
    'responsive': True,
}

BASE_STYLE = dict(
    template='plotly_white',
    width=1250,
    height=700,
    font=dict(size=14, family='Arial', color='#111827'),
    margin=dict(l=70, r=40, t=95, b=70),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='left',
        x=0.0,
        title_text='Curve',
        font=dict(size=13),
        title_font=dict(size=13),
        bordercolor='#D1D5DB',
        borderwidth=1,
        itemclick='toggleothers',
        itemdoubleclick='toggle',
    ),
)

THRESHOLD_P = {
    10: 900,
    12: 3100,
    14: 11500,
    16: 50000,
}

def repo_root() -> Path:
    cwd = Path.cwd().resolve()
    candidates = [cwd, cwd.parent, cwd.parent.parent]
    for c in candidates:
        if (c / 'results').exists() and (c / 'notebooks').exists():
            return c
    raise RuntimeError('Repository root non trovata')

ROOT = repo_root()
RES = ROOT / 'results'

print('ROOT =', ROOT)


ROOT = /Users/daniele/CLionProjects/satp-cpp


In [2]:

def load_stream_pair(k: int):
    hll_path = RES / 'HyperLogLog' / f'k_{k}_L_32' / 'results_streaming.csv'
    hllpp_path = RES / 'HyperLogLog++' / f'k_{k}' / 'results_streaming.csv'
    if not hll_path.exists() or not hllpp_path.exists():
        return None

    h = pd.read_csv(hll_path)
    p = pd.read_csv(hllpp_path)

    keys = ['sample_size', 'distinct_count', 'seed', 'runs', 'element_index']
    m = h.merge(p, on=keys, suffixes=('_hll', '_hllpp'))
    m['k'] = k

    m['abs_rel_hll'] = (m['f0_hat_mean_hll'] - m['f0_mean_hll']).abs() / m['f0_mean_hll'].replace(0, np.nan)
    m['abs_rel_hllpp'] = (m['f0_hat_mean_hllpp'] - m['f0_mean_hllpp']).abs() / m['f0_mean_hllpp'].replace(0, np.nan)
    m['gain'] = m['abs_rel_hll'] - m['abs_rel_hllpp']

    m['ratio'] = (m['distinct_count'] / m['sample_size']).round(4)

    return m

parts = [load_stream_pair(k) for k in [10, 12, 14, 16]]
parts = [p for p in parts if p is not None]
merged = pd.concat(parts, ignore_index=True)

print('rows:', len(merged))
print('k disponibili:', sorted(merged['k'].unique()))
print('sample_size:', sorted(merged['sample_size'].unique()))
print('seed:', sorted(merged['seed'].unique()))


rows: 11200
k disponibili: [np.int64(10), np.int64(12), np.int64(14), np.int64(16)]
sample_size: [np.int64(10000), np.int64(100000), np.int64(1000000), np.int64(10000000)]
seed: [np.int64(21041998)]


In [3]:

# Endpoint summary: dove HLL++ guadagna di più
end = merged[merged['element_index'] == merged['sample_size']].copy()

summary_endpoint = (
    end.groupby(['k', 'sample_size', 'distinct_count'], as_index=False)
       .agg(
           abs_rel_hll=('abs_rel_hll', 'mean'),
           abs_rel_hllpp=('abs_rel_hllpp', 'mean'),
           gain=('gain', 'mean'),
           f0_true=('f0_mean_hll', 'mean'),
           f0_hat_hll=('f0_hat_mean_hll', 'mean'),
           f0_hat_hllpp=('f0_hat_mean_hllpp', 'mean'),
       )
)

summary_endpoint['ratio'] = summary_endpoint['distinct_count'] / summary_endpoint['sample_size']
summary_endpoint = summary_endpoint.sort_values('gain', ascending=False)

print('Top 15 endpoint (gain positivo = HLL++ migliore):')
print(summary_endpoint.head(15).to_string(index=False))

print('Casi endpoint dove HLL è leggermente migliore (gain < 0):')
print(summary_endpoint[summary_endpoint['gain'] < 0].sort_values('gain').head(15).to_string(index=False))


Top 15 endpoint (gain positivo = HLL++ migliore):
 k  sample_size  distinct_count  abs_rel_hll  abs_rel_hllpp     gain  f0_true  f0_hat_hll  f0_hat_hllpp  ratio
10       100000          100000     0.032240       0.000050 0.032190 100000.0   103224.00     100005.00   1.00
10        10000           10000     0.021900       0.000300 0.021600  10000.0     9781.00       9997.00   1.00
12      1000000           10000     0.021108       0.000054 0.021054  10000.0    10211.08       9999.46   0.01
12       100000           10000     0.017474       0.000066 0.017408  10000.0    10174.74       9999.34   0.10
12       100000          100000     0.017360       0.000050 0.017310 100000.0   101736.00     100005.00   1.00
14       100000           50000     0.015410       0.000026 0.015385  50000.0    50770.52      50001.28   0.50
14       100000          100000     0.013460       0.000050 0.013410 100000.0   101346.00     100005.00   1.00
12       100000           50000     0.009676       0.000026 0.

In [4]:

def style_axes(fig, x_title='Elementi processati (t)', y_title='Valore'):
    fig.update_layout(**BASE_STYLE)
    fig.update_xaxes(title=x_title, showgrid=True, gridcolor='#E5E7EB', fixedrange=False, rangeslider=dict(visible=True, thickness=0.07))
    fig.update_yaxes(title=y_title, showgrid=True, gridcolor='#E5E7EB', fixedrange=False)


def plot_estimate_scenario(n, d, k, seed=21041998):
    sdf = merged[
        (merged['sample_size'] == n)
        & (merged['distinct_count'] == d)
        & (merged['k'] == k)
        & (merged['seed'] == seed)
    ].sort_values('element_index')

    if sdf.empty:
        print(f'Nessun dato per n={n}, d={d}, k={k}, seed={seed}')
        return

    m = 2 ** k
    th_25m = int(2.5 * m)
    th_p = THRESHOLD_P.get(k)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=sdf['element_index'], y=sdf['f0_mean_hll'], mode='lines',
        name='F0 reale', line=dict(color='#111827', width=3)
    ))
    fig.add_trace(go.Scatter(
        x=sdf['element_index'], y=sdf['f0_hat_mean_hll'], mode='lines',
        name=f'HLL·k={k}', line=dict(color='#1D4ED8', width=2.8)
    ))
    fig.add_trace(go.Scatter(
        x=sdf['element_index'], y=sdf['f0_hat_mean_hllpp'], mode='lines',
        name=f'HLL++·k={k}', line=dict(color='#059669', width=2.8)
    ))

    for xval, label, color in [
        (th_25m, f'2.5m={th_25m}', '#DC2626'),
        (th_p, f'Threshold(p)={th_p}', '#7C3AED') if th_p is not None else (None, None, None),
    ]:
        if xval is not None and xval <= n:
            fig.add_vline(x=xval, line_width=1.5, line_dash='dash', line_color=color)
            fig.add_annotation(
                x=xval, y=1.0, xref='x', yref='paper',
                text=label, showarrow=False, yshift=10,
                font=dict(size=12, color=color), bgcolor='rgba(255,255,255,0.85)'
            )

    style_axes(fig, y_title='Cardinalità stimata')
    fig.update_layout(title=f'Scenario: n={n:,}, d={d:,}, k={k} (seed={seed})')
    fig.show(config=PLOT_CONFIG)


def plot_error_scenario(n, d, k, seed=21041998):
    sdf = merged[
        (merged['sample_size'] == n)
        & (merged['distinct_count'] == d)
        & (merged['k'] == k)
        & (merged['seed'] == seed)
    ].sort_values('element_index')

    if sdf.empty:
        print(f'Nessun dato per n={n}, d={d}, k={k}, seed={seed}')
        return

    m = 2 ** k
    th_25m = int(2.5 * m)
    th_p = THRESHOLD_P.get(k)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=sdf['element_index'], y=sdf['abs_rel_hll'], mode='lines',
        name=f'|rel err| HLL·k={k}', line=dict(color='#1D4ED8', width=2.8)
    ))
    fig.add_trace(go.Scatter(
        x=sdf['element_index'], y=sdf['abs_rel_hllpp'], mode='lines',
        name=f'|rel err| HLL++·k={k}', line=dict(color='#059669', width=2.8)
    ))
    fig.add_trace(go.Scatter(
        x=sdf['element_index'], y=sdf['gain'], mode='lines',
        name='Gain = err(HLL)-err(HLL++)', line=dict(color='#D97706', width=2.2, dash='dot')
    ))

    for xval, label, color in [
        (th_25m, f'2.5m={th_25m}', '#DC2626'),
        (th_p, f'Threshold(p)={th_p}', '#7C3AED') if th_p is not None else (None, None, None),
    ]:
        if xval is not None and xval <= n:
            fig.add_vline(x=xval, line_width=1.5, line_dash='dash', line_color=color)

    style_axes(fig, y_title='Errore relativo assoluto / gain')
    fig.update_layout(title=f'Errore relativo e gain: n={n:,}, d={d:,}, k={k} (seed={seed})')
    fig.show(config=PLOT_CONFIG)


In [5]:

# Scenari con massima divergenza (gain elevato)
# 1) caso forte ad alta cardinalità relativa
plot_estimate_scenario(n=100_000, d=100_000, k=10)
plot_error_scenario(n=100_000, d=100_000, k=10)

# 2) caso forte small-range/intermedio
plot_estimate_scenario(n=1_000_000, d=10_000, k=12)
plot_error_scenario(n=1_000_000, d=10_000, k=12)


In [6]:

# Caso grande n=10^7: qui HLL e HLL++ tendono ad allinearsi (stesse stime quasi ovunque)
for d in [100_000, 1_000_000, 5_000_000, 10_000_000]:
    plot_error_scenario(n=10_000_000, d=d, k=10)
    plot_error_scenario(n=10_000_000, d=d, k=12)


In [7]:

# Heatmap endpoint del gain (HLL - HLL++), separata per k
end2 = summary_endpoint.copy()

ks = sorted(end2['k'].unique())
fig = make_subplots(rows=1, cols=len(ks), subplot_titles=[f'k={k}' for k in ks])

for i, k in enumerate(ks, start=1):
    sdf = end2[end2['k'] == k].copy()
    if sdf.empty:
        continue
    pivot = sdf.pivot_table(index='distinct_count', columns='sample_size', values='gain', aggfunc='mean')
    pivot = pivot.sort_index().sort_index(axis=1)

    fig.add_trace(
        go.Heatmap(
            z=pivot.values,
            x=[f'n={int(x):,}' for x in pivot.columns],
            y=[f'd={int(y):,}' for y in pivot.index],
            colorscale='RdYlGn',
            zmid=0.0,
            colorbar=dict(title='gain', x=1.02 if i == len(ks) else None),
            hovertemplate='n=%{x}<br>d=%{y}<br>gain=%{z:.6f}<extra></extra>',
            showscale=(i == len(ks)),
        ),
        row=1, col=i
    )

fig.update_layout(
    template='plotly_white',
    width=1500,
    height=650,
    title='Endpoint gain = |rel err HLL| - |rel err HLL++| (positivo = HLL++ migliore)',
    font=dict(size=14),
    margin=dict(l=70, r=80, t=90, b=70),
)
fig.show(config=PLOT_CONFIG)



## Lettura rapida dei risultati

- `gain > 0`: HLL++ ha errore relativo assoluto inferiore a HLL.
- Le divergenze più nette emergono in alcuni regimi intermedi (`n=10^5, d=10^5, k=10` e `n=10^6, d=10^4, k=12`).
- Nel caso molto grande `n=10^7` (con `k=10,12`) le due curve sono quasi sovrapposte: in quel regime la differenza è piccola ma ancora misurabile.
